In [1]:
using StatisticalRethinking, Distributed, JLD, LinearAlgebra
using Mamba

Data

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [3]:
df2 = filter(row -> row[:age] >= 18, df);

Input data for Mamba

In [4]:
data = Dict(
  :x => convert(Array{Float64,1}, df2[:weight]),
  :y => convert(Array{Float64,1}, df2[:height])
);

Log-transformed Posterior(b0, b1, log(s2)) + Constant and Gradient Vector

In [5]:
logfgrad = function(x::DenseVector)
  b0 = x[1]
  b1 = x[2]
  logs2 = x[3]
  r = data[:y] .- b0 .- b1 .* data[:x]
  logf = (-0.5 * length(data[:y]) - 0.001) * logs2 -
           (0.5 * dot(r, r) + 0.001) / exp(logs2) -
           0.5 * b0^2 / 1000 - 0.5 * b1^2 / 1000
  grad = [
    sum(r) / exp(logs2) - b0 / 1000,
    sum(data[:x] .* r) / exp(logs2) - b1 / 1000,
    -0.5 * length(data[:y]) - 0.001 + (0.5 * dot(r, r) + 0.001) / exp(logs2)
  ]
  logf, grad
end

# MCMC Simulation with No-U-Turn Sampling

n = 5000
burnin = 1000
sim = Mamba.Chains(n, 3, start = (burnin + 1), names = ["b0", "b1", "s2"])
theta = NUTSVariate([0.0, 0.0, 0.0], logfgrad)
for i in 1:n
  sample!(theta, adapt = (i <= burnin))
  if i > burnin
    sim[i, :, 1] = [theta[1:2]; exp(theta[3])]
  end
end

Summarize draws

In [6]:
describe(sim)#-

Iterations = 1001:5000
Thinning interval = 1
Chains = 1
Samples per chain = 4000

Empirical Posterior Estimates:
       Mean        SD        Naive SE       MCSE         ESS   
b0 113.4289443 1.94034557 0.03067955721 0.1123284635  298.38606
b1   0.9148116 0.04264046 0.00067420486 0.0024304954  307.78976
s2  26.0941680 1.95288131 0.03087776463 0.0474376278 1694.75050

Quantiles:
       2.5%       25.0%        50.0%       75.0%        97.5%   
b0 109.6462386 112.1091181 113.51942899 114.7862550 117.07371972
b1   0.8359927   0.8844868   0.91409193   0.9440536   0.99758164
s2  22.6234003  24.7097104  26.04172102  27.3623292  30.12961770



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*